In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catalyst

In [3]:
import os

from pathlib import Path

import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from PIL import Image
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

from catalyst import dl
from catalyst.dl.callbacks import AccuracyCallback, EarlyStoppingCallback

In [4]:
ROOT_DIR = '/content/drive/My Drive/Colab Notebooks/'
TRAIN_SAMPLES_PATH = 'train.csv'
VAL_SAMPLES_PATH = 'val.csv'
SUBMISSION_FILE = 'submission.csv'
MODEL_PATH = 'model.pkt'
NUM_CLASSES = 38
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cuda


In [5]:
class MultiClassDataset(Dataset):

    def __init__(self , csv_file , img_dir , transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
    
    def __getitem__(self, idx):
        d = self.df.iloc[idx]
        image = Image.open(f'{self.img_dir}/{d.Name}').convert("RGB")
        label = torch.tensor(d[1:].tolist() , dtype=torch.float32)
    
        if self.transform is not None:
            image = self.transform(image)
        return image, label
  
    def __len__(self):
        return len(self.df)

In [6]:
df = pd.read_csv(ROOT_DIR + 'data/training_labels.csv')
df_train, df_val = train_test_split(df, test_size = 0.1)

df_train.to_csv(TRAIN_SAMPLES_PATH, index=False)
df_val.to_csv(VAL_SAMPLES_PATH, index=False)

In [7]:
batch_size = 64

transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip()                                 
])

transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

training_set = MultiClassDataset(TRAIN_SAMPLES_PATH , ROOT_DIR + 'data/training_images' , transform_train)
validation_set = MultiClassDataset(VAL_SAMPLES_PATH , ROOT_DIR + 'data/training_images' , transform_val)

print(f'Number of samples: Train: {len(training_set)}, Validation: {len(validation_set)}')
loaders = {
    "train":DataLoader(training_set , shuffle=True, batch_size=batch_size),
    "valid": DataLoader(validation_set , shuffle=False, batch_size=batch_size)
    }

Number of samples: Train: 3370, Validation: 375


In [8]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def get_classification_network(in_features, num_classes):
    return nn.Sequential(
         nn.Linear(in_features, in_features // 2),
         nn.BatchNorm1d(in_features // 2),
         nn.ReLU(),
         nn.Dropout(p = 0.5),
         nn.Linear(in_features // 2, in_features // 4),
         nn.BatchNorm1d(in_features // 4),
         nn.ReLU(),
         nn.Dropout(p = 0.5),
         nn.Linear(in_features // 4, in_features // 8),
         nn.BatchNorm1d(in_features // 8),
         nn.ReLU(),
         nn.Dropout(p = 0.5),
         nn.Linear(in_features // 8, num_classes)
    )

def get_model(num_classes, feature_extract=True):
    model = models.resnext101_32x8d(pretrained = True)
    set_parameter_requires_grad(model, feature_extract)
    num_ftrs = model.fc.in_features
    model.fc = get_classification_network(num_ftrs, num_classes)
    return model

def skyhacks_f1_score(preds, y):
    return f1_score(y, preds, average = 'macro')

In [9]:
model = get_model(NUM_CLASSES)

In [10]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
callbacks = [
    EarlyStoppingCallback(patience=5, metric = 'f1-score', minimize = False)
]
num_epochs = 100

class CustomRunner(dl.Runner):
    
    def predict_batch(self, batch):
        x, y = batch
        return self.model(x.to(self.device))
    
    def _handle_batch(self, batch):
        x, y = batch
        output = self.model(x)
        loss = self.state.criterion(output, y)

        preds = torch.sigmoid(output).data > 0.5
        preds = preds.to(torch.float32)

        f1 = skyhacks_f1_score(preds.cpu().numpy(), y.cpu().numpy())
        self.batch_metrics = {
            "loss": loss,
            "f1-score": f1
        }
        
        if self.state.is_train_loader:
            loss.backward()
            self.state.optimizer.step()
            self.state.optimizer.zero_grad()           


In [11]:
runner = CustomRunner()

runner.train(
    loaders=loaders,
    model=model, 
    criterion=criterion, 
    optimizer=optimizer,
    callbacks=callbacks,
    num_epochs=num_epochs, 
    logdir="./logs", 
    verbose=True
)

1/100 * Epoch (train):  17% 9/53 [00:32<02:40,  3.65s/it, f1-score=0.135, loss=0.466]

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.



1/100 * Epoch (train): 100% 53/53 [03:19<00:00,  3.77s/it, f1-score=0.142, loss=0.233]
1/100 * Epoch (valid): 100% 6/6 [00:17<00:00,  2.83s/it, f1-score=0.192, loss=0.209]
[2020-11-14 18:14:09,994] 
1/100 * Epoch 1 (train): f1-score=0.1156 | loss=0.3245
1/100 * Epoch 1 (valid): f1-score=0.1852 | loss=0.2118
2/100 * Epoch (train): 100% 53/53 [03:19<00:00,  3.77s/it, f1-score=0.145, loss=0.190]
2/100 * Epoch (valid): 100% 6/6 [00:16<00:00,  2.79s/it, f1-score=0.268, loss=0.180]
[2020-11-14 18:18:11,952] 
2/100 * Epoch 2 (train): f1-score=0.1708 | loss=0.2069
2/100 * Epoch 2 (valid): f1-score=0.2386 | loss=0.1877
3/100 * Epoch (train): 100% 53/53 [03:16<00:00,  3.71s/it, f1-score=0.250, loss=0.205]
3/100 * Epoch (valid): 100% 6/6 [00:16<00:00,  2.75s/it, f1-score=0.279, loss=0.172]
[2020-11-14 18:22:13,646] 
3/100 * Epoch 3 (train): f1-score=0.2060 | loss=0.1941
3/100 * Epoch 3 (valid): f1-score=0.2633 | loss=0.1790
4/100 * Epoch (train): 100% 53/53 [03:17<00:00,  3.72s/it, f1-score=0.235

In [12]:
torch.save(runner.model, MODEL_PATH)
files.download(MODEL_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>